In [ ]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import pyodbc
import pandas.rpy.common as com
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
#import sqlite3
# supervised model for labels

In [ ]:
# Load the data

# df = pd.read_csv('WAKE_COUNTY_HOUSING_DATA.csv')

# conn = pyodbc.connect('Driver={SQL Server};'
#                       'Server=MSI;'
#                       'Database=DATAPORTFOLIO;'
#                       'Trusted_Connection=yes;')

conn = sqlite3.connect('housingSQL.db')
query = "SELECT * FROM housing_table"
homes_df = pd.read_sql_query(query, conn)
conn.close()

sql_query = pd.read_sql_query('''
                              select * from DATAPORTFOLIO.dbo.WAKE_COUNTY_HOUSING_DATA
                              '''
                              ,conn) # here, the 'conn' is the variable that contains your database connection

df = pd.DataFrame(sql_query)

df.head()

# logistic is binary, k means is clusters (consumers based on income)

In [ ]:
# Check data types of all columns
df.dtypes

In [ ]:
# https://stackoverflow.com/questions/39409866/correlation-heatmap
# Create a correlation heatmap to assess features for their significance
# load the R package ISLR
# infert = com.importr("ISLR")

# # load the Auto dataset
# df = com.load_data('Homes')

# # calculate the correlation matrix
# corr = df.corr()

# # plot the heatmap
# sns.heatmap(corr, 
#         xticklabels=corr.columns,
#         yticklabels=corr.columns)

Var_Corr = df.corr()
# plot the heatmap and annotation on it
sns.heatmap(Var_Corr, xticklabels=Var_Corr.columns, yticklabels=Var_Corr.columns, annot=True)

In [ ]:
# Select features we're interested in
features = ['HEATEDAREA', 'AIR_NOISE', 'TOTAL_CRIMES', 'POP_DENSITY', 'DESIGN_STYLE', 'YEAR_BUILT', 'ZIP_CODE', 'CITY']
X = homes_df[features]

In [ ]:
# 'TOTAL_VALUE' home cost is our target variable
y = df['TOTAL_VALUE']

In [ ]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Create a Linear Regression model
home_value_model = LinearRegression()

In [ ]:
# Train the model
home_value_model.fit(X_train, y_train)

In [ ]:
# You can now use the model to make predictions, for example:
y_pred = home_value_model.predict(X_test)

In [2]:
# Add predictions to dataframe
homes_df['prediction'] = y_pred

In [ ]:
# Define your bins edges
bins = [0, 250000, 500000, float('inf')]

In [ ]:
# Define labels
value_labels = ['<250k', '250-500k', '>500k']

In [ ]:
# Create a new column with categorical labels
homes_df['price_range'] = pd.cut(homes_df['prediction'], bins=bins, labels=value_labels)